# CNN
***

## 概要  
人間の視覚の仕組みを取り入れ成功したモデルである。現在も広く使われており、その応用範囲はとても広い。画像認識、クラス分類、など画像が絡むタスクはだいたいCNNを用いて解決される. 

## 用語説明  
***
### CNN

畳込み層を用いるニューラルネットワークのこと。

### 画像

デジタル画像には、カラー画像とモノクロ(グレースケール)画像の二種類がある。カラー画像はRGBでピクセルを制御している。モノクロ画像は0~255の値で離散化して持つ

### 畳込み演算
画像など(一般にtensor)に対して、各要素ごとにフィルタを積和演算する演算のこと。以下の用語説明や、理論解説でも取り扱う。

![画像]()

### 畳込み層
畳込み演算を行う層を畳込み層と呼ぶ。英語ではConvolution Layer

### 局所受容野、カーネル、フィルター

説明する人の出身領域で言葉が変わってくるが本質的には同じものを指す。神経科学系なら局所受容野、画像系ならフィルター、その他はカーネルと呼んでいる人が多い気がする。今後、この資料ではフィルタでと呼ぶ。

### パディング

畳込みを行うと、tensorの大きさが小さくなるから、大きさを調整するために0などで画像の周囲を埋める

### ストライド

フィルタをどれだけ動かすか。通常は1

### 重み共有

### プーリング

図のような演算をプーリング(max pooling)という。

### プーリング層

プーリングを行う層をプーリング層という。この層を挟むことによって、フィルターの感度が鈍くなることになる。その結果、位置ずれに対する頑健性があがると言われている。一方で、特徴が失われるという否定的な意見もある。

## 理論解説
***
$$
u_{ij}=\sum_{p=0}^{H-1}\sum_{q=0}^{H-1}\,\it{x}_{i+p\, j+q}\,h_{pq}\hspace{2em}\tag{1}
$$

## 実装解説
***


In [1]:
# CNNの実装。MNISTの分類をするか
import torch 
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
from torchvision.datasets import MNIST
from torchvision import transforms
from tqdm import tqdm

In [3]:
mnist_train = MNIST("./Dataset/04/MNIST",
                   train=True, download=True,
                   transform=transforms.ToTensor())

mnist_test = MNIST("./Dataset/04/MNIST",
                  train=False, download=True,
                  transform=transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(mnist_train,
                         batch_size=batch_size, shuffle=True)

test_loader = DataLoader(mnist_test,
                        batch_size=batch_size, shuffle=False)


 18%|████████████                                                        | 1753088/9912422 [00:05<00:24, 333872.74it/s]

KeyboardInterrupt: 

In [ ]:
# (N, C, H, W) 形式のTensorを(N, C*H*W) に引き伸ばす
# 畳込み層の出力をMLPに渡す際に用いる
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)
    
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    
    FlattenLayer()
)

test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200,10)
)

net = nn.Sequential(
    conv_net,
    mlp
)

In [ ]:
def eval_net(net, data_loader, device = 'cpu'):
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            _, y_pred = torch.max(net(x), 1)
        ys.append(y)
        ypreds.append(y_pred)
        
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    
    acc = (ys == ypreds).float().sum() / len (ys)
    return acc.item()
    
def train_net(net, train_loader, test_loader,
             #optimizer = optim.Adam,
             loss_func = nn.CrossEntropyLoss(),
             n_iter = 10, device = 'cpu'):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optim.Adam(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.
        net.train()
        n = 0
        n_acc = 0
        
        for i, (xx, yy) in tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = torch.max(h, 1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        
        train_acc.append(n_acc / n)
        
        val_acc.append(eval_net(net, test_loader, device))
        
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)
            


In [ ]:
#net.to('cuda:0')
train_net(net, train_loader, test_loader, n_iter= 20)

演習問題